# **Prepare directories:**

In [12]:
import os
import shutil
import random

def prepare_brats_data(folder_path, abbreviation, sample_size):

    if not os.path.isdir(folder_path):
        raise ValueError(f"Directory does not exist: {folder_path}")
    
    folder_path = os.path.abspath(folder_path)
    base_folder_name = os.path.basename(folder_path)
    output_root = os.path.join(os.path.dirname(folder_path), f"{base_folder_name}_processed")

    segmentation_dir = os.path.join(output_root, "segmentation")
    input_dir = os.path.join(output_root, "input")
    output_dir = os.path.join(output_root, "output")
    all_dir = os.path.join(output_root, "all")

    os.makedirs(segmentation_dir, exist_ok=True)
    os.makedirs(input_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(all_dir, exist_ok=True)

    all_patients = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    if len(all_patients) < sample_size:
        raise ValueError(f"Too few patients: found {len(all_patients)}, but {sample_size} were requested.")

    chosen_patients = random.sample(all_patients, sample_size)
    print(f"Chose {sample_size} patients from {len(all_patients)} available.")

    modality_map = {
        "t1n": "t1",
        "t1c": "t1ce",
        "t2w": "t2",
        "t2f": "flair"
    }

    for patient_folder in chosen_patients:
        full_path = os.path.join(folder_path, patient_folder)
        parts = patient_folder.split("-")
        if len(parts) < 4:
            print(f"Skipped due to bad directory name: {patient_folder}")
            continue

        patient_id = parts[2]

        for raw_mod, new_mod in modality_map.items():
            src = os.path.join(full_path, f"{patient_folder}-{raw_mod}.nii.gz")
            dst = os.path.join(all_dir, f"BraTS{abbreviation}_{patient_id}_{new_mod}.nii.gz")
            if os.path.exists(src):
                shutil.copy(src, dst)
            else:
                print(f"File: {src} does not exist.")

        seg_src = os.path.join(full_path, f"{patient_folder}-seg.nii.gz")
        seg_dst = os.path.join(segmentation_dir, f"BraTs{abbreviation}_{patient_id}_seg.nii.gz")
        if os.path.exists(seg_src):
            shutil.copy(seg_src, seg_dst)
        else:
            print(f"Segmentation file {seg_src} is missing.")
    
    print(f"Processing complete. Files are available in '{output_root}'.")

# BraTS 2023

In [15]:
prepare_brats_data(
    folder_path=r"C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023",
    abbreviation="2023",
    sample_size=20
)

Chose 20 patients from 1251 available.
Processing complete. Files are available in 'C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed'.


# BraTS Africa

In [16]:
prepare_brats_data(
    folder_path=r"C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa",
    abbreviation="Africa",
    sample_size=20
)

Chose 20 patients from 60 available.
Processing complete. Files are available in 'C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed'.


# **Run Segmentation**

In [ ]:
import os
import shutil
import subprocess
from tqdm import tqdm

def empty_directory(target_dir):
    for entry in os.listdir(target_dir):
        entry_path = os.path.join(target_dir, entry)
        if os.path.isfile(entry_path):
            os.remove(entry_path)

def perform_segmentation_on_folder(base_folder):
    all_dir = os.path.join(base_folder, "all")
    in_dir = os.path.join(base_folder, "input")
    out_dir = os.path.join(base_folder, "output")

    unique_ids = set()
    for filename in os.listdir(all_dir):
        split_name = filename.split("_")
        if len(split_name) > 1:
            unique_ids.add(split_name[1])
    sorted_ids = sorted(unique_ids)

    empty_directory(in_dir)

    progress_bar = tqdm(sorted_ids, desc=f"Segmenting: {os.path.basename(base_folder)}",
                        bar_format="{l_bar}%s{bar}%s{r_bar}" % ("\033[32m", "\033[0m"), ncols=80)

    for case_id in progress_bar:
        related_files = [fname for fname in os.listdir(all_dir) if case_id in fname]
        for fname in related_files:
            shutil.copy2(os.path.join(all_dir, fname), os.path.join(in_dir, fname))

        subprocess.run([
            "docker", "run", "--rm", "--gpus", "all",
            "-v", f"{in_dir}:/input",
            "-v", f"{out_dir}:/output",
            "rixez/brats21nnunet"
        ])

        empty_directory(in_dir)

def main():
    processed_folders = [
        r"/home/fylypo/WB/BraTs_2023_processed",
        r"/home/fylypo/WB/BraTs_Africa_processed"
    ]

    for folder in processed_folders:
        if not os.path.isdir(folder):
            print(f"Folder not found: {folder}")
            continue
        perform_segmentation_on_folder(folder)

if __name__ == "__main__":
    main()

Directory: /home/fylypo/WB/BraTs_2023_processed does not exist.


# **Change Labels**

In [1]:
import nibabel as nib
import os

def change_label_4_to_3(output_dir, abbreviation):
    for filename in os.listdir(output_dir):
        if filename.endswith(".nii.gz"):
            file_path = os.path.join(output_dir, filename)
            pred = nib.load(file_path)
            data = pred.get_fdata()
            data[data == 4] = 3

            new_seg = nib.Nifti1Image(data, pred.affine, pred.header)
            new_filename = f"BraTS{abbreviation}_{filename}"
            new_file_path = os.path.join(output_dir, new_filename)
            nib.save(new_seg, new_file_path)
            os.remove(file_path)
            print(f"Changed label 4 to 3 and renamed {file_path} to {new_file_path}")

# BraTS 2023

In [4]:
change_label_4_to_3(r"C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output", "2023")

Changed label 4 to 3 and renamed C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output\00035.nii.gz to C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output\BraTS2023_00035.nii.gz
Changed label 4 to 3 and renamed C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output\00194.nii.gz to C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output\BraTS2023_00194.nii.gz
Changed label 4 to 3 and renamed C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output\00214.nii.gz to C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output\BraTS2023_00214.nii.gz
Changed label 4 to 3 and renamed C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output\00348.nii.gz to C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output\BraTS2023_00348.nii.gz
Changed label 4 to 3 and renamed C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output\00373.nii.gz to C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output\BraTS2023_00373.nii.gz
Changed la

# BraTS Africa

In [5]:
change_label_4_to_3(r"C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output", "Africa")

Changed label 4 to 3 and renamed C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output\00007.nii.gz to C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output\BraTSAfrica_00007.nii.gz
Changed label 4 to 3 and renamed C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output\00010.nii.gz to C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output\BraTSAfrica_00010.nii.gz
Changed label 4 to 3 and renamed C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output\00028.nii.gz to C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output\BraTSAfrica_00028.nii.gz
Changed label 4 to 3 and renamed C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output\00050.nii.gz to C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output\BraTSAfrica_00050.nii.gz
Changed label 4 to 3 and renamed C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output\00051.nii.gz to C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output\BraTS

# **Calculate Dice Score: WARNING SOMETHING IS BROKEN**

In [ ]:
import numpy as np
import nibabel as nib
import os

def compute_brats_metrics(pred, label):
    results = {}
    regions = {
        'ET': ([3], [3]),
        'TC': ([2, 3], [2, 3]),
        'WT': ([1, 2, 3], [1, 2, 3])
    }

    for name, (pred_classes, label_classes) in regions.items():
        y_pred = np.isin(pred, pred_classes).astype(np.uint8)
        y_true = np.isin(label, label_classes).astype(np.uint8)

        eps = 1e-7
        tp = np.sum(y_true * y_pred)
        fp = np.sum(y_pred) - tp
        fn = np.sum(y_true) - tp

        results[name] = {
            'Dice': (2 * tp + eps) / (2 * tp + fp + fn + eps),
            'Sensitivity': tp / (tp + fn + eps),
            'Specificity': np.sum((1 - y_true) * (1 - y_pred)) / (np.sum(1 - y_true) + eps),
            'Volume_diff': np.abs(np.sum(y_pred.astype(np.int64)) - np.sum(y_true.astype(np.int64)))
        }

    for label_val in [1, 2, 3]:
        y_pred = (pred == label_val).astype(np.uint8)
        y_true = (label == label_val).astype(np.uint8)
        eps = 1e-7
        tp = np.sum(y_true * y_pred)
        fp = np.sum(y_pred) - tp
        fn = np.sum(y_true) - tp
        dice = (2 * tp + eps) / (2 * tp + fp + fn + eps)
        results[f"Label_{label_val}"] = {'Dice': dice}

    return results

def evaluate(labels_path, predictions_path):
    print("Evaluating folder...")

    label_files = sorted([f for f in os.listdir(labels_path) if f.endswith('.nii') or f.endswith('.nii.gz')])
    pred_files = sorted([f for f in os.listdir(predictions_path) if f.endswith('.nii') or f.endswith('.nii.gz')])

    brats_results = []

    for label_file, pred_file in zip(label_files, pred_files):
        label_img = nib.load(os.path.join(labels_path, label_file))
        pred_img = nib.load(os.path.join(predictions_path, pred_file))
        label = label_img.get_fdata()
        pred = pred_img.get_fdata()
        case_metrics = compute_brats_metrics(pred, label)
        brats_results.append(case_metrics)

    region_names = ['ET', 'TC', 'WT']
    label_names = ['Label_1', 'Label_2', 'Label_3']

    region_summary = {}
    for region in region_names:
        dice_vals = [r[region]['Dice'] for r in brats_results]
        sensitivity_vals = [r[region]['Sensitivity'] for r in brats_results]
        specificity_vals = [r[region]['Specificity'] for r in brats_results]
        vol_diff_vals = [r[region]['Volume_diff'] for r in brats_results]

        region_summary[region] = {
            'Dice Mean': round(np.mean(dice_vals), 3),
            'Dice Std': round(np.std(dice_vals), 3),
            'Sensitivity': round(np.mean(sensitivity_vals), 3),
            'Specificity': round(np.mean(specificity_vals), 3),
            'Mean Volume Diff': round(np.mean(vol_diff_vals), 1)
        }

    label_summary = {}
    for label_name in label_names:
        dice_vals = [r[label_name]['Dice'] for r in brats_results]
        label_summary[label_name] = {
            'Dice Mean': round(np.mean(dice_vals), 3),
            'Dice Std': round(np.std(dice_vals), 3)
        }

    print("\nComposite Region Metrics (WT, TC, ET):")
    for region, values in region_summary.items():
        print(f"\n{region}:")
        for k, v in values.items():
            print(f"  {k}: {v}")

    print("\nDice Scores for Individual Labels (1, 2, 3):")
    for label_name, values in label_summary.items():
        print(f"  {label_name}:")
        for k, v in values.items():
            print(f"    {k}: {v}")

# BraTS 2023

In [6]:
evaluate(
    r"C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\segmentation",
    r"C:\Users\szlin\Desktop\Studia\WB 2\BraTs_2023_processed\output"
)


Evaluating folder...

Composite Region Metrics (WT, TC, ET):

ET:
  Dice Mean: 0.0
  Dice Std: 0.0
  Sensitivity: 0.0
  Specificity: 1.0
  Mean Volume Diff: 21952.2

TC:
  Dice Mean: 0.0
  Dice Std: 0.0
  Sensitivity: 0.0
  Specificity: 1.0
  Mean Volume Diff: 64477.6

WT:
  Dice Mean: 0.0
  Dice Std: 0.0
  Sensitivity: 0.0
  Specificity: 1.0
  Mean Volume Diff: 79263.4

Dice Scores for Individual Labels (1, 2, 3):
  Label_1:
    Dice Mean: 0.0
    Dice Std: 0.0
  Label_2:
    Dice Mean: 0.0
    Dice Std: 0.0
  Label_3:
    Dice Mean: 0.0
    Dice Std: 0.0


# BraTS Africa

In [7]:
evaluate(
    r"C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\segmentation",
    r"C:\Users\szlin\Desktop\Studia\WB 2\BraTs_Africa_processed\output"
)

Evaluating folder...

Composite Region Metrics (WT, TC, ET):

ET:
  Dice Mean: 0.0
  Dice Std: 0.0
  Sensitivity: 0.0
  Specificity: 1.0
  Mean Volume Diff: 42729.0

TC:
  Dice Mean: 0.0
  Dice Std: 0.0
  Sensitivity: 0.0
  Specificity: 1.0
  Mean Volume Diff: 150004.6

WT:
  Dice Mean: 0.0
  Dice Std: 0.0
  Sensitivity: 0.0
  Specificity: 1.0
  Mean Volume Diff: 187068.8

Dice Scores for Individual Labels (1, 2, 3):
  Label_1:
    Dice Mean: 0.05
    Dice Std: 0.218
  Label_2:
    Dice Mean: 0.0
    Dice Std: 0.0
  Label_3:
    Dice Mean: 0.0
    Dice Std: 0.0
